# Slackbot Example

SlackBot keeps you in the loop without disturbing your focus. Its personalized, intelligent AI continuously monitors your Slack workspace, alerting you to important conversations and freeing you to concentrate on what’s most important.

SlackBot reads the full history of your (public) Slack workspace and trains a Generative AI model to predict when you need to engage with a conversation. This training process gives the AI a deep understanding of your interests, expertise, and relationships. Using this understanding, SlackBot watches conversations in real-time and notifies you when an important conversation is happening without you. With SlackBot200 you can focus on getting things done without worrying about missing out.

In this notebook, you’ll see you how to build and deploy SlackBot in 15 minutes using only OpenAI’s API’s and open-source Python libraries - Data Science PhD not required.



In [ ]:
%pip install openai kaskada

In [ ]:
from datetime import datetime, timedelta
from slack_sdk.socket_mode import SocketModeClient
from slack_sdk.socket_mode.response import SocketModeResponse
import sparrow_py as kt
import pandas
import openai
import getpass
import pyarrow
import datetime

# Initialize Kaskada with a local execution context.
kt.init_session()

# Initialize OpenAI
openai.api_key = getpass.getpass('OpenAI: API Key')

## Fine-tune the model

### Read Historical Messages

In [ ]:
messages = kt.sources.ArrowSource(
    data = pandas.read_parquet("./messages.parquet"), 
    time_column_name = "ts", 
    key_column_name = "channel",
)


messages.preview(5)

### Construct prompts

In [ ]:
messages = messages.with_key(kt.record({
        "channel": messages.col("channel"),
        "thread": messages.col("thread_ts"),
    }))

prompts = messages \
    .select("user", "ts", "text", "reactions") \
    .collect(max=20)

prompts.preview(5)

### Build examples

In [ ]:
duration = datetime.timedelta(minutes=5)

shifted_conversation = conversations.shift_by(duration)
reaction_users = messages.col("reactions").col("users").flatten().collect(max=100).flatten()
#reaction_users = messages.col("reactions").flatten().col("users").collect(kt.Trailing(duration)).flatten()
#participating_users = conversations.col("user").collect(max=100) #kt.windows.Trailing(duration))
engaged_users = reaction_users #kt.union(reaction_users, participating_users)

#examples = kt.record({"prompt": shifted_conversation, "completion": engaged_users}) \
#    .filter(shifted_conversation.is_not_null())
examples = kt.record({"prompt": conversations, "completion": engaged_users})
examples.preview(5) # NOTE: completion shouldn't be None

## Fine-tune a model

### Create training dataset

In [ ]:
from sklearn import preprocessing

examples_df = examples.run().to_pandas().drop(["_time", "_subsort", "_key_hash", "_key"], axis=1)

le = preprocessing.LabelEncoder()
le.fit(examples_df.completion.explode())

# Format for the OpenAI API
def format_prompt(prompt):
    return "start -> " + "\n\n".join([f' {msg["user"]} --> {msg["text"]} ' for msg in prompt]) + "\n\n###\n\n"
examples_df.prompt = examples_df.prompt.apply(format_prompt)

def format_completion(completion):
    return " " + (" ".join(le.transform(completion).astype(str)) if len(completion) > 0 else "nil") + " end"
examples_df.completion = examples_df.completion.apply(format_completion)

# Write examples to file
examples_df.to_json("examples.jsonl", orient='records', lines=True)

### Upload to OpenAI

In [ ]:
from types import SimpleNamespace
from openai import cli

# verifiy data format, split for training & validation
args = SimpleNamespace(file='./examples.jsonl', quiet=True)
cli.FineTune.prepare_data(args)
training_id = cli.FineTune._get_or_upload('./examples_prepared_train.jsonl', True)

### Create the training job

In [ ]:
import openai

resp = openai.FineTune.create(
    training_file = training_id,
    model = "davinci",
    n_epochs = 2,
    learning_rate_multiplier = 0.02,
    suffix = "coversation_users"
)
print(f'Fine-tuning model with job ID: "{resp["id"]}"')